In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

Напишем необходимые для сбора данных функции.

In [2]:
def get_page_content(URL):
    """Функция для отправления запроса на страницу и парсинга BeautifulSoup"""
    response = requests.get(URL)
    if not response.ok:
        return response
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [3]:
def get_genres_list(response_content):
    """Функция для получения ссылок на страницы жанров"""
    genres = []
    obj = response_content.find('ul', attrs = {'class':'zhanr'})

    for item in obj.children:
        try:    
            for inner_item in item.children:
                genres.append(inner_item.get("href"))
        except AttributeError:
            pass
    return genres

In [4]:
def get_links_list(response_content):
    """Функция для получения списка ссылок на страницы фильмов"""
    obj = response_content.find('div', attrs = {'class':'art content'})
    content = obj.find_all('a')

    page_list = []
    for item in content:
        if item.startswith('https://doramy.club/'):
            try:
                href = item.get("href")
                if '/genre/' not in href:
                    page_list. append(href)
            except AttributeError:
                pass
        
    return page_list

In [6]:
def get_final_content(response_content):
    """Функция, отвечающая за сбор данных со страницы"""
    obj = response_content.find('tbody')
    properties = obj.find_all('td')
    
    total_series = '1 серия'
    episod_duration = '-'
    year = '-'
    country = '-'
    genres = '-'
    translation = '-'
    list_properties = [i.text for i in properties]
    for index in range(len(list_properties)):
        if 'Просмотр:' in list_properties[index]:
            episod_duration = list_properties[index + 1]
        elif 'Сериал:' in list_properties[index]:
            total_series = list_properties[index + 1]
        elif 'Год:' in list_properties[index]:
            year = list_properties[index + 1]
        elif 'Страна:' in list_properties[index]:
            country = list_properties[index + 1]
        elif 'Жанр:' in list_properties[index]:
            genres = list_properties[index + 1]
        elif 'Перевод:' in list_properties[index]:
            translation = list_properties[index + 1]
            
    name = response_content.find('h1', attrs = {'itemprop':'name'}).text
    score = float(response_content.find('span', attrs = {'itemprop':'ratingValue'}).text)
    description = response_content.find('div', attrs = {'class':'annotaciya'}).text.replace('\n', ' ').strip('\n')

    data = '!;!'.join([name, episod_duration, str(total_series), year, country,
            genres, translation, str(score), description])
    
    return data

In [7]:
def create_data_file(file_name):
    """Инициализирует пустой файл для сбора данных"""
    columns = '!;!'.join(['Name', 'Episod_duration', 'Total_series', 'Year',
                          'Country', 'Genres', 'Translation', 'Score', 'Description\n'])
    with open(file_name, 'w') as file:
        file.write(columns)
        
        
def add_info_to_DF(file_name, data):
    """Добаление записи в файл данных"""
    with open(file_name, 'a') as file:
        file.write(data)
        

def get_last_page_number(response_content):
    """Получение номера последней страницы на странице жанра"""
    obj = response_content.find('div', attrs = {'class':'art content'})
    content = obj.find_all('a')  
    return int(content[-2].text)

In [22]:
def get_all_films(links_list, data, pause):
    """Функция для прохода по списку ссылок на страницы фильмов"""
    for link in links:
        if link.startswith('https://doramy.club/') and 'page' not in link:
            time.sleep(pause)
            print('Скачиваем', link)
            URL = link
            source = get_page_content(URL)
            try:
                string = get_final_content(source)
            except Exception:
                print('\n','-'*10)
                print('Проблема со скачиванием:', link)
                print('\n','-'*10)
                pass
            data += string + '\n'
    return data

Соберем данные: название фильма, год, страну, количество серий и их продолжительность, перевод, описание, жанр и рейтинг.

In [10]:
page = get_page_content('https://doramy.club/')
genres_list = get_genres_list(page)

In [23]:
pause = 3
file_name = 'results.csv'
create_data_file(file_name)
for genre in genres_list:
    time.sleep(pause)
    print('\nНачинаем скачивать', genre)
    URL = 'https://doramy.club/'[:-1] + genre
    source = get_page_content(URL)    
    last_page_number = get_last_page_number(source)
    links = get_links_list(source)

    data = ''
    data = get_all_films(links, data, pause)
    for page_number in range(2, last_page_number + 1):  
        time.sleep(pause)
        print('Скачиваем страницу', page_number, '\n')
        URL = 'https://doramy.club/'[:-1] + genre + '/page/{}'.format(page_number)
        source = get_page_content(URL)
        links = get_links_list(source)
        data = get_all_films(links, data, pause)

        if page_number % 5 == 0:
            print('-'*10)
            add_info_to_DF(file_name, data)
            data = ''


Начинаем скачивать /genre/muzyka
Скачиваем https://doramy.club/37707-moyo-dvadcatoe-dvadcatiletie.html
Скачиваем https://doramy.club/37041-sekretnaya-fan-missiya.html
Скачиваем https://doramy.club/37008-i-land.html
Скачиваем https://doramy.club/34177-pochemu-ty-i-ya.html
Скачиваем https://doramy.club/35695-zhizn-prekrasna.html
Скачиваем https://doramy.club/35454-bojbend.html
Скачиваем https://doramy.club/10993-miss-babulya.html
Скачиваем https://doramy.club/33053-ulichnye-tancy-kitaya-5.html
Скачиваем https://doramy.club/34047-lyubov-zdes-zapreshhena.html
Скачиваем https://doramy.club/33603-ulybnis-zhizni.html
Скачиваем страницу 2 

Скачиваем https://doramy.club/33004-o-yunosti.html
Скачиваем https://doramy.club/19824-nashi-yarkie-dni.html
Скачиваем https://doramy.club/32529-novyj-shans-spet.html
Скачиваем https://doramy.club/32621-nodame-kantabile.html
Скачиваем https://doramy.club/32497-pochti-znamenity.html
Скачиваем https://doramy.club/32049-bud-moim-parnem-2-roza-v-dome.html
Скач

Скачиваем https://doramy.club/34675-odnazhdy-v-vysshej-shkole.html
Скачиваем https://doramy.club/34194-shkola-fantomov.html
Скачиваем https://doramy.club/34086-mest-drugix.html
Скачиваем страницу 3 

Скачиваем https://doramy.club/31447-shkolnyj-krasavchik.html
Скачиваем https://doramy.club/33493-sezony-cveteniya.html
Скачиваем https://doramy.club/31450-kogda-ty-budesh-mnoj.html
Скачиваем https://doramy.club/33872-devushka-dvadcatogo-veka.html
Скачиваем https://doramy.club/32564-podrazhatel.html
Скачиваем https://doramy.club/33634-lyubovnaya-lixoradka-2019.html
Скачиваем https://doramy.club/33490-ty-moya-nedostayushhaya-polovinka.html
Скачиваем https://doramy.club/33546-tak-molody.html
Скачиваем https://doramy.club/33463-durackij-poceluj.html
Скачиваем https://doramy.club/33375-drug-na-nedelyu.html
Скачиваем страницу 4 

Скачиваем https://doramy.club/33086-vysshij-ball.html
Скачиваем https://doramy.club/19824-nashi-yarkie-dni.html
Скачиваем https://doramy.club/28174-moment-siyaniya-serd

Скачиваем https://doramy.club/17783-druzheskij-kontrakt.html
Скачиваем https://doramy.club/7994-cvetok-i-zver-cvetok-zverya.html
Скачиваем https://doramy.club/17922-magazinchik-vedmy.html
Скачиваем https://doramy.club/17893-genij.html
Скачиваем https://doramy.club/17843-iskushenie-volkov.html
----------
Скачиваем страницу 16 

Скачиваем https://doramy.club/17705-35-letnyaya-shkolnica.html
Скачиваем https://doramy.club/17661-master-obucheniya.html
Скачиваем https://doramy.club/17649-rostki-lyubvi.html
Скачиваем https://doramy.club/15976-nan-aj.html
Скачиваем https://doramy.club/10048-dragonzakura.html
Скачиваем https://doramy.club/10093-reka-grusti.html
Скачиваем https://doramy.club/15895-neudacha-v-lyubvi.html
Скачиваем https://doramy.club/15922-chyornyj-pyos.html
Скачиваем https://doramy.club/17045-princ-iz-legend-2.html
Скачиваем https://doramy.club/16077-final-kotoryj-my-iskali.html
Скачиваем страницу 17 

Скачиваем https://doramy.club/16005-privet-zhu-xua.html
Скачиваем https://dor

Скачиваем https://doramy.club/6971-prodolzhenie-sleduet.html
Скачиваем https://doramy.club/6014-uchitel.html
Скачиваем https://doramy.club/6807-starshaya-shkola-bakaleya-film.html
Скачиваем https://doramy.club/6699-starshaya-shkola-bakaleya.html
Скачиваем https://doramy.club/6585-zatknis-i-igraj.html
Скачиваем страницу 29 

Скачиваем https://doramy.club/6516-klass-korolevy-korejskaya-versiya.html
Скачиваем https://doramy.club/1786-vnimanie-lyubov.html
Скачиваем https://doramy.club/6397-pozvol-mne-lyubit-tebya.html
Скачиваем https://doramy.club/6307-tajny-shkolnogo-dvora.html
Скачиваем https://doramy.club/4232-strastnoe-serdce.html
Скачиваем https://doramy.club/6057-korol-starshej-shkoly.html
Скачиваем https://doramy.club/5878-cvetochki-posle-yagodok-2-vozvrashhenie.html
Скачиваем https://doramy.club/5874-cvetochki-posle-yagodok-yaponiya.html
Скачиваем https://doramy.club/4319-bezumnyj-azart.html
Скачиваем https://doramy.club/5570-ozornoj-poceluj-tajskaya-versiya.html
Скачиваем страницу

Скачиваем https://doramy.club/27576-intuiciya.html
Скачиваем https://doramy.club/23343-bezmolvnyj-prestupnik.html
Скачиваем https://doramy.club/15816-zapoluchit-eyo.html
Скачиваем https://doramy.club/27143-oxota-na-monstra-2.html
Скачиваем https://doramy.club/27139-oxota-na-monstra.html
Скачиваем https://doramy.club/13334-vozvrashhenie-princa-yan-lin-v-min.html
Скачиваем https://doramy.club/26961-zolotoj-monax.html
Скачиваем https://doramy.club/26928-lis-i-uchenyj.html
Скачиваем страницу 7 

Скачиваем https://doramy.club/26903-iz-pekina-v-moskvu.html
Скачиваем https://doramy.club/26846-assasin-bitva-mirov.html
Скачиваем https://doramy.club/25483-1-pridvornaya-dama.html
Скачиваем https://doramy.club/25249-6-puteshestviya-change.html
Скачиваем https://doramy.club/26300-sem-mechej-cvetok-semi-chuvstv.html
Скачиваем https://doramy.club/24143-carstvo-duxov.html
Скачиваем https://doramy.club/26263-zateryannaya-grobnica-perezagruzka-budda-iz-zmeinoj-kosti.html
Скачиваем https://doramy.club/25

...

Скачиваем https://doramy.club/28402-vozrozhdenie-dlya-tebya.html
Скачиваем https://doramy.club/27584-para-zerkal.html
Скачиваем https://doramy.club/29708-chetyre-damy.html
Скачиваем https://doramy.club/15049-gulyaka-iz-prigranichnogo-goroda.html
Скачиваем https://doramy.club/10833-proshhaj-moya-princessa.html
Скачиваем страницу 25 

Скачиваем https://doramy.club/28821-krasnyj-manzhet-rukava.html
Скачиваем https://doramy.club/28751-skazanie-o-tajnom-korolevskom-inspektore-i-cho-i.html
Скачиваем https://doramy.club/27042-vanton.html
Скачиваем https://doramy.club/12877-basketbol.html
Скачиваем https://doramy.club/889-legenda-o-chu.html
Скачиваем https://doramy.club/19526-osennyaya-cikada.html
Скачиваем https://doramy.club/22088-zagadka-genov-lisy.html
Скачиваем https://doramy.club/28036-molodoj-master-cinchen.html
Скачиваем https://doramy.club/28293-vlyublennost-2021.html
Скачиваем https://doramy.club/27949-pravosudie-2021.html
----------
Скачиваем страницу 26 

Скачиваем https://doramy.c

Скачиваем https://doramy.club/21191-laon-dao.html
Скачиваем https://doramy.club/22127-vyjdi-za-menya-zamuzh.html
Скачиваем https://doramy.club/24098-zapredelnost.html
Скачиваем https://doramy.club/16554-milaya-devushka.html
Скачиваем https://doramy.club/1006-spisok-lanya-2.html
Скачиваем https://doramy.club/11663-nebo-dlya-odnoj-zvezdy.html
Скачиваем https://doramy.club/10957-legendy.html
Скачиваем https://doramy.club/19857-2-moya-dorogaya-sudba.html
Скачиваем страницу 38 

Скачиваем https://doramy.club/23781-raskrashennaya-kozha-3.html
Скачиваем https://doramy.club/23761-ubijstvo-yujven-xu.html
Скачиваем https://doramy.club/23493-detektiv-2020.html
Скачиваем https://doramy.club/23702-ya-prikryvayu-imperatora.html
Скачиваем https://doramy.club/9151-legenda-o-nefritovom-meche.html
Скачиваем https://doramy.club/12106-1-legenda-o-fenikse.html
Скачиваем https://doramy.club/17389-oxota-na-shpiona.html
Скачиваем https://doramy.club/13215-devyat-carstv-znamya-orla-novolandiya-znamya-orla.html

Скачиваем https://doramy.club/15685-radost-zhizni.html
Скачиваем https://doramy.club/15849-vybor-vojny-mezhdu-devushkami.html
Скачиваем https://doramy.club/1728-kovarnaya-nechist-mogila-laski.html
Скачиваем https://doramy.club/9401-chosonskij-mag.html
Скачиваем https://doramy.club/5342-ledi-i-lzhec.html
Скачиваем https://doramy.club/8953-vostochnaya.html
Скачиваем https://doramy.club/16117-korolevskij-portnoj.html
Скачиваем https://doramy.club/9994-lyubov-v-xanyuan.html
Скачиваем https://doramy.club/14853-tigr-iz-roda-drakonov.html
Скачиваем https://doramy.club/15732-pravitel-marionetka.html
----------
Скачиваем страницу 51 

Скачиваем https://doramy.club/5858-1-oxotniki-na-rabov.html
Скачиваем https://doramy.club/14509-moya-strana.html
Скачиваем https://doramy.club/15646-zapiski-ob-olene-i-trenozhnike.html
Скачиваем https://doramy.club/13625-princessa-vejyan.html
Скачиваем https://doramy.club/15534-pismo-na-den-rozhdeniya.html
Скачиваем https://doramy.club/13965-skazka-o-nok-du.html
С

Скачиваем https://doramy.club/4845-legenda-o-kote-demone.html
Скачиваем https://doramy.club/1085-vrata-kunlun-proshloe-i-nastoyashhee.html
Скачиваем https://doramy.club/3569-derevo-s-glubokimi-kornyami.html
Скачиваем https://doramy.club/3828-plemena-i-imperii.html
Скачиваем https://doramy.club/4558-tysyacha-lic-dunczya.html
Скачиваем https://doramy.club/445-1-legenda-sinego-morya.html
Скачиваем https://doramy.club/4279-chetvero-3.html
Скачиваем https://doramy.club/4274-chetvero-2.html
Скачиваем https://doramy.club/4264-chetvero.html
Скачиваем страницу 64 

Скачиваем https://doramy.club/1073-v-plenu-lyubvi.html
Скачиваем https://doramy.club/3493-krasota-v-trudnye-vremena.html
Скачиваем https://doramy.club/3559-shest-letyashhix-drakonov.html
Скачиваем https://doramy.club/1012-xmelnoj-zvon.html
Скачиваем https://doramy.club/3902-podmennye-vojska.html
Скачиваем https://doramy.club/3815-chan-ok-chon-zhizn-radi-lyubvi.html
Скачиваем https://doramy.club/3811-taksist.html
Скачиваем https://dor

Скачиваем https://doramy.club/15715-1-liga-pechi.html
Скачиваем https://doramy.club/15991-tancy-v-stile-slem.html
Скачиваем https://doramy.club/13384-pogonya-za-myachom.html
Скачиваем https://doramy.club/4963-silnejshie.html
Скачиваем страницу 8 

Скачиваем https://doramy.club/12441-silnyj-veter.html
Скачиваем https://doramy.club/14869-chelovek-chesti.html
Скачиваем https://doramy.club/10863-lstec.html
Скачиваем https://doramy.club/12447-ego-istoriya-2.html
Скачиваем https://doramy.club/6947-gordost.html
Скачиваем https://doramy.club/11029-sokrushitelnyj-udar-2.html
Скачиваем https://doramy.club/10996-zabit-na-poslednej-sekunde.html
Скачиваем https://doramy.club/4829-short-trek.html
Скачиваем https://doramy.club/9439-tancovshhicy.html
Скачиваем https://doramy.club/7672-romantika-bez-pravil.html
Скачиваем страницу 9 

Скачиваем https://doramy.club/7419-sokrushitelnyj-udar.html
Скачиваем https://doramy.club/6929-chempion.html
Скачиваем https://doramy.club/6645-restler.html
Скачиваем http

В ходе работы возникли неполадки и функцию пришлось перезапустить. Поэтому в результате было получено 2 файла данных. Соберем их воедино.

In [26]:
data_01 = pd.read_csv('results_01.csv', sep='!;!')
data_02 = pd.read_csv('results_02.csv', sep='!;!')

/tmp/ipykernel_66125/445144356.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_01 = pd.read_csv('results_01.csv', sep='!;!')
/tmp/ipykernel_66125/445144356.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_02 = pd.read_csv('results_02.csv', sep='!;!')


In [25]:
data.head(100)

,Name,Episod_duration,Total_series,Year,Country,Genres,Translation,Score,Description
0,Моё двадцатое двадцатилетие дорама (2023),15 мин,12 серий,2023,Южная Корея,"Музыка, Фантастика",Русские субтитры,8.67,"Отправляясь встречать новый год с друзьями, С..."
1,Секретная фан-миссия дорама (2023),35 мин,10 серий,2023,Таиланд,"Комедия, Музыка",Русские субтитры,6.93,Secret Dream — самая популярная группа мальчи...
2,I-LAND дорама (2020),1 час 18 мин.,12,2020,Южная Корея,"Музыка, Ток-шоу",Русские субтитры,9.80,"Музыкальное шоу на выживание, где 23 участник..."
3,Почему ты и я? дорама (2022),45 мин,10 серий,2022,Таиланд,"Комедия, Мелодрама, Музыка",Русские субтитры,8.85,"Гэп и Намва учатся в одном университете, а ещ..."
4,Жизнь прекрасна дорама (2023),2 часа,1 серия,2023,Южная Корея,"Драма, Музыка",Русская озвучка,9.00,Фильм рассказывает о женщине по имени О Сэ Ëн...
...,...,...,...,...,...,...,...,...,...
95,Нэиль Кантабиле дорама (2014),1 час,16 серий,2014,Южная Корея,"Комедия, Мелодрама, Музыка","Русская озвучка, Русские субтитры",8.20,Главный герой дорамы «Нэиль Кантабиле» Чха Ю ...
96,Слишком прекрасна для меня дорама (2014),1 час,16 серий,2014,Южная Корея,"Мелодрама, Музыка","Русская озвучка, Русские субтитры",8.55,События романтического сериала «Слишком прекр...
97,Шоумен дорама (2016),60 мин.,18 серий,2016,Южная Корея,"Драма, Комедия, Мелодрама, Музыка","Русская озвучка, Русские субтитры",8.92,"Син Сок Хо, шоумен, ему довелось работать мен..."
98,Тайный роман дорама (2014),1 час,16 серий,2014,Южная Корея,"Мелодрама, Музыка","Русская озвучка, Русские субтитры",8.72,События романтического сериала «Тайный роман»...


In [27]:
summ_data = pd.concat([data_01, data_02], axis=0)

In [29]:
summ_data.to_csv('data.csv', index=False)

In [31]:
pd.read_csv('data.csv').head(10)

,Name,Episod_duration,Total_series,Year,Country,Genres,Translation,Score,Description
0,Странная легенда династии Тан дорама (2022),45 мин,36 серий,2022,Китай,"Детектив, Исторический, Фэнтези","Русская озвучка, Русские субтитры",9.04,Странные дела происходят в древней столице Ча...
1,Письмо с небес дорама (2022),44 мин,26 серий,2022,Китай,"Детектив, Исторический, Комедия, Мелодрама",Русские субтитры,9.39,"Молодая, но уже довольно опытная в жизни Шен ..."
2,Тринадцать лет забвения дорама (2023),44 мин,24 серии,2023,Китай,"Детектив, Криминал, Триллер",Русские субтитры,9.31,Тринадцать лет назад в городе Наньду произошл...
3,Шах и мат дорама (2022),45 мин,24 серии,2022,Китай,"Детектив, Исторический, Триллер","Русская озвучка, Русские субтитры",8.96,Молодой любитель юриспруденции и расследовани...
4,Домашняя школа: ученики под арестом дорама (2023),50 мин,18 серий,2023,Таиланд,"Детектив, Триллер","Русская озвучка, Русские субтитры",9.16,"В специальный интернат, заявленный, как домаш..."
5,Три дня до выпускного дорама (2022),15 мин,24 серии,2022,Япония,Детектив,"Русская озвучка, Русские субтитры",6.45,Четверым ученикам старшей школы пришло письмо...
6,Союз благородных дорама (2023),48 мин,29 серий,2023,Китай,"Детектив, Исторический, Приключения, Фэнтези","Русская озвучка, Русские субтитры",9.00,Сюжет дорамы расскажет о молодом ученом Чжан ...
7,Быть героем дорама (2022),42 мин,32 серии,2022,Китай,"Детектив, Криминал","Русская озвучка, Русские субтитры",8.98,Главный герой Чэнь Юй борется за справедливос...
8,Древняя песня о любви дорама (2023),30 мин,14 серий,2023,Китай,"Детектив, Драма, Исторический, Мелодрама, Фант...",Русские субтитры,10.00,Профессор и автор книг по истории современнос...
9,Что скрывают маски дорама (2019),45 мин.,55 серий,2019,Китай,"Детектив, Исторический, Мелодрама","Русская озвучка, Русские субтитры",8.75,Главная героиня дорамы «Что скрывают маски» с...
